# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

# EXPLAIN ENTIRE PROJECT HERE, and evaluation **rubrics**

In [ ]:
import numpy as np
import os
# from scipy.ndimage import imread, imresize
from skimage.io import imread
from skimage.transform import resize as imresize
import datetime

We set the random seed so that the results don't vary drastically.

In [ ]:
# !pip install ipdb

In [ ]:
# # !pip install tensorflow==2.8.0
# import keras
# print(keras.__version__)

In [ ]:
np.random.seed(30)

import random as rn
rn.seed(30)

from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

1. read the folder names for training and validation.
2. set the `batch_size` here such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_doc = np.random.permutation(open('/content/drive/MyDrive/Colab Notebooks/gesture_recognition/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/drive/MyDrive/Colab Notebooks/gesture_recognition/Project_data/val.csv').readlines())
batch_size = 51

## Generator
1. preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames.
2. experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [11,12,13,14,15,16,17,18,19,20,21,22,23,24,25] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(source_path)/batch_size) # calculate the number of batches
        # print('num_batch', num_batches)
        # print(len(source_path))
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,15,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                # print("folder_name", t[folder + (batch*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    # print('source_path',source_path, '  folder', folder, ' batch', batch, 'bactch size',batch_size)
                    # print("len of img", len(imgs), "item:", item)
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image.resize(100,100)
                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] /= 255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] /= 255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] /= 255 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
        if((len(source_path)%batch_size)//2==0):
            batch_size = 2
        else:
            batch_size = 1
        num_batches = len(source_path)%batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,15,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    # Cropped image of above dimension
                    # (It will not change orginal image)

                    #image = image.crop((0, 0, 120, 120))
                    image = image.resize(100, 100)


                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] /= 255
                    batch_data[folder,idx,:,:,1] /= 255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] /= 255 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do



Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/drive/MyDrive/Colab Notebooks/gesture_recognition/Project_data/train'
val_path = '/content/drive/MyDrive/Colab Notebooks/gesture_recognition/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 15
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 15


In [ ]:
len(train_path)

77

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

Input_shape = (15,100,100,3)
model = Sequential()
model.add(Conv3D(32, (3,3,3), padding='same',input_shape=Input_shape))
#model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=sample_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(32, (3, 3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2,2)))
model.add(Dropout(0.5))

model.add(Conv3D(64, (3, 3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(64, (3, 3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
optimiser = tf.keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 15, 100, 100, 32   2624      
                             )                                   
                                                                 
 activation (Activation)     (None, 15, 100, 100, 32   0         
                             )                                   
                                                                 
 batch_normalization (Batch  (None, 15, 100, 100, 32   128       
 Normalization)              )                                   
                                                                 
 conv3d_1 (Conv3D)           (None, 13, 98, 98, 32)    27680     
                                                                 
 activation_1 (Activation)   (None, 13, 98, 98, 32)    0         
                                                        

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

# filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
filepath = model_name + 'model-epoch{epoch:05d}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto') #, save_freq=1

LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 10,
  verbose = 0, mode = "auto", min_delta = 1e-04, cooldown = 0,
  min_lr = 0)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
num_train_sequences

663

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/drive/MyDrive/Colab Notebooks/gesture_recognition/Project_data/train ; batch size = 51
Epoch 1/15
13/13 [==============================] - ETA: 0s - loss: 1.6094 - categorical_accuracy: 0.1429Source path =  /content/drive/MyDrive/Colab Notebooks/gesture_recognition/Project_data/val ; batch size = 51

Epoch 1: saving model to model_init_2024-03-0311_01_43.968933/model-epoch00001.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


13/13 [==============================] - 773s 62s/step - loss: 1.6094 - categorical_accuracy: 0.1429 - val_loss: 1.6095 - val_categorical_accuracy: 0.1923 - lr: 0.0010
Epoch 2/15
13/13 [==============================] - ETA: 0s - loss: 1.6086 - categorical_accuracy: 0.2308
Epoch 2: saving model to model_init_2024-03-0311_01_43.968933/model-epoch00002.h5
13/13 [==============================] - 129s 11s/step - loss: 1.6086 - categorical_accuracy: 0.2308 - val_loss: 1.6039 - val_categorical_accuracy: 0.5000 - lr: 0.0010
Epoch 3/15
13/13 [==============================] - ETA: 0s - loss: 1.6112 - categorical_accuracy: 0.0000e+00 
Epoch 3: saving model to model_init_2024-03-0311_01_43.968933/model-epoch00003.h5
13/13 [==============================] - 115s 10s/step - loss: 1.6112 - categorical_accuracy: 0.0000e+00 - val_loss: 1.6082 - val_categorical_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 4/15
13/13 [==============================] - ETA: 0s - loss: 1.6093 - categorical_accuracy: 0.2308 


Model CONV2D + LSTM

In [ ]:
#write your model here
from keras.models import Sequential, Model
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, LSTM, Bidirectional


Input_shape_1 = (18, 100, 100, 3)
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2), activation='relu', padding='same'), input_shape=Input_shape_1))
model.add(TimeDistributed(Conv2D(32, (3,3), kernel_initializer="he_normal", activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(512, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(512, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Flatten()))

model.add(Dropout(0.5))
model.add(LSTM(512, return_sequences=False, dropout=0.5))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 18, 50, 50, 32)    4736      
 ributed)                                                        
                                                                 
 time_distributed_1 (TimeDi  (None, 18, 48, 48, 32)    9248      
 stributed)                                                      
                                                                 
 time_distributed_2 (TimeDi  (None, 18, 24, 24, 32)    0         
 stributed)                                                      
                                                                 
 time_distributed_3 (TimeDi  (None, 18, 24, 24, 64)    18496     
 stributed)                                                      
                                                                 
 time_distributed_4 (TimeDi  (None, 18, 24, 24, 64)   

In [ ]:
#optimiser = Keras.optimizers.Adam(lr=0.001)
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 18, 50, 50, 32)    4736      
 ributed)                                                        
                                                                 
 time_distributed_1 (TimeDi  (None, 18, 48, 48, 32)    9248      
 stributed)                                                      
                                                                 
 time_distributed_2 (TimeDi  (None, 18, 24, 24, 32)    0         
 stributed)                                                      
                                                                 
 time_distributed_3 (TimeDi  (None, 18, 24, 24, 64)    18496     
 stributed)                                                      
                                                                 
 time_distributed_4 (TimeDi  (None, 18, 24, 24, 64)   

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'Gesture_recog' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-model-epoch{epoch:05d}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 10,
  verbose = 0, mode = "auto", epsilon = 1e-04, cooldown = 0,
  min_lr = 0)
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
history  = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-22-bd77c9c60c14>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  /content/drive/MyDrive/Colab Notebooks/gesture_recognition/Project_data/train ; batch size = 51
Epoch 1/15
13/13 [==============================] - ETA: 0s - loss: 1.6093 - categorical_accuracy: 0.1746Source path =  /content/drive/MyDrive/Colab Notebooks/gesture_recognition/Project_data/val ; batch size = 51

Epoch 1: saving model to Gesture_recog_2024-03-0311_01_43.968933/model-model-epoch00001.h5
13/13 [==============================] - 266s 21s/step - loss: 1.6093 - categorical_accuracy: 0.1746 - val_loss: 1.6151 - val_categorical_accuracy: 0.1923 - lr: 0.0100
Epoch 2/15
 1/13 [=>............................] - ETA: 0s - loss: 1.6132 - categorical_accuracy: 0.0000e+00

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


13/13 [==============================] - ETA: 0s - loss: 1.6155 - categorical_accuracy: 0.2308
Epoch 2: saving model to Gesture_recog_2024-03-0311_01_43.968933/model-model-epoch00002.h5
13/13 [==============================] - 70s 6s/step - loss: 1.6155 - categorical_accuracy: 0.2308 - val_loss: 1.6441 - val_categorical_accuracy: 0.0000e+00 - lr: 0.0100
Epoch 3/15
13/13 [==============================] - ETA: 0s - loss: 1.6207 - categorical_accuracy: 0.0769    
Epoch 3: saving model to Gesture_recog_2024-03-0311_01_43.968933/model-model-epoch00003.h5
13/13 [==============================] - 58s 4s/step - loss: 1.6207 - categorical_accuracy: 0.0769 - val_loss: 1.6108 - val_categorical_accuracy: 0.5000 - lr: 0.0100
Epoch 4/15
13/13 [==============================] - ETA: 0s - loss: 1.6122 - categorical_accuracy: 0.2308
Epoch 4: saving model to Gesture_recog_2024-03-0311_01_43.968933/model-model-epoch00004.h5
13/13 [==============================] - 118s 9s/step - loss: 1.6122 - categoric

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(20,6))
# ax1 = plt.subplot(121)
# ax1 = plt.plot(history.history['loss'])
# ax1 = plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='lower left')
# ax2 = plt.subplot(122)
# ax2 = plt.plot(history.history['categorical_accuracy'])
# ax2 = plt.plot(history.history['val_categorical_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('categorical_accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='lower left')